In [33]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from numpy import mean,std
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,recall_score
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.naive_bayes import GaussianNB

## Loading Data

In [34]:
data = data = pd.read_csv('../Datasets/telecommunications_churn.csv',sep=';')
data.head()

,account_length,voice_mail_plan,voice_mail_messages,day_mins,evening_mins,night_mins,international_mins,customer_service_calls,international_plan,day_calls,day_charge,evening_calls,evening_charge,night_calls,night_charge,international_calls,international_charge,total_charge,churn
0,128,1,25,265.1,197.4,244.7,10.0,1,0,110,45.07,99,16.78,91,11.01,3,2.70,75.56,0
1,107,1,26,161.6,195.5,254.4,13.7,1,0,123,27.47,103,16.62,103,11.45,3,3.70,59.24,0
2,137,0,0,243.4,121.2,162.6,12.2,0,0,114,41.38,110,10.30,104,7.32,5,3.29,62.29,0
3,84,0,0,299.4,61.9,196.9,6.6,2,1,71,50.90,88,5.26,89,8.86,7,1.78,66.80,0
4,75,0,0,166.7,148.3,186.9,10.1,3,1,113,28.34,122,12.61,121,8.41,3,2.73,52.09,0


## Scaling Data and Seperating X and y

In [35]:
std_scr = StandardScaler()
X = data.drop(labels='churn',axis=1)
cols = X.columns
X = pd.DataFrame(std_scr.fit_transform(X))
X.columns = cols
y = data[['churn']]
X.shape,y.shape

((3333, 18), (3333, 1))

## Train Test Split

In [36]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=12,stratify=y)

### Dict to record model performance

In [37]:
my_dict = {'model':[],'train accuracy':[], 'test accuracy': [],'precision':[],'recall':[]}

### Function to pridict and record model performance

In [38]:
def populate_matrix(fittedModel):
    y_pred_train = fittedModel.predict(x_train)
    train_accuracy = round(accuracy_score(y_train,y_pred_train)*100,2)
    
    y_pred = fittedModel.predict(x_test)
    test_accuracy = round(accuracy_score(y_test,y_pred)*100,2)
    precision = round(precision_score(y_test,y_pred)*100,2)
    recall = round(recall_score(y_test,y_pred)*100,2)
    model = str(fittedModel)
    my_dict["model"].append(model)
    my_dict["train accuracy"].append(train_accuracy)
    my_dict["test accuracy"].append(test_accuracy)
    my_dict["precision"].append(precision)
    my_dict["recall"].append(recall)


In [39]:
def cross_validate_model(model):
    # evaluate model
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    n_scores = cross_val_score(estimator=model,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

## Model Validation

## Logistic Model

In [40]:
# evaluate model
logistic = LogisticRegression()
cross_validate_model(logistic)


Accuracy: 0.861 (0.009)


### Model Training

In [41]:
logistic.fit(x_train,y_train)
populate_matrix(logistic)
print(my_dict)

{'model': ['LogisticRegression()'], 'train accuracy': [86.01], 'test accuracy': [86.51], 'precision': [56.6], 'recall': [30.93]}


## Decision Tree

In [43]:
dtree = DecisionTreeClassifier()
# evaluate model
cross_validate_model(dtree)

Accuracy: 0.950 (0.011)


### Model Training

In [44]:
dtree.fit(x_train,y_train)
populate_matrix(dtree)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()'], 'train accuracy': [86.01, 100.0], 'test accuracy': [86.51, 93.25], 'precision': [56.6, 72.81], 'recall': [30.93, 85.57]}


## RandomForest

In [45]:
rf = RandomForestClassifier()
# evaluate model
cross_validate_model(rf)

Accuracy: 0.977 (0.007)


### Model Training

In [46]:
rf.fit(x_train,y_train)
populate_matrix(rf)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()'], 'train accuracy': [86.01, 100.0, 100.0], 'test accuracy': [86.51, 93.25, 97.3], 'precision': [56.6, 72.81, 98.77], 'recall': [30.93, 85.57, 82.47]}


## Gradient Boosting

In [47]:
gboost = GradientBoostingClassifier()
# evaluate model
cross_validate_model(gboost)

Accuracy: 0.978 (0.007)


### Model Training

In [48]:
gboost.fit(x_train,y_train)
populate_matrix(gboost)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()'], 'train accuracy': [86.01, 100.0, 100.0, 98.42], 'test accuracy': [86.51, 93.25, 97.3, 97.75], 'precision': [56.6, 72.81, 98.77, 98.81], 'recall': [30.93, 85.57, 82.47, 85.57]}


## SVM

In [49]:
svm = SVC()
# evaluate model
cross_validate_model(svm)

Accuracy: 0.928 (0.011)


### Model Training

In [50]:
svm.fit(x_train,y_train)
populate_matrix(svm)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()'], 'train accuracy': [86.01, 100.0, 100.0, 98.42, 95.31], 'test accuracy': [86.51, 93.25, 97.3, 97.75, 92.8], 'precision': [56.6, 72.81, 98.77, 98.81, 88.89], 'recall': [30.93, 85.57, 82.47, 85.57, 57.73]}


## ANN

In [51]:
ann_model = Sequential()
ann_model.add(Dropout(0.30))
ann_model.add(Dense(20,input_dim=x_train.shape[1],activation='relu'))
ann_model.add(Dense(10,activation='relu'))
ann_model.add(Dropout(0.30))
ann_model.add(Dense(5,activation='relu'))
ann_model.add(Dense(1,activation='sigmoid'))


In [52]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


### Model Training

In [53]:
ann_model.fit(x_train,y_train,batch_size=100,epochs=200,validation_data=(x_test,y_test),shuffle=True)



Epoch 1/200
27/27 [==============================] - 1s 6ms/step - loss: 0.7252 - accuracy: 0.4321 - val_loss: 0.6742 - val_accuracy: 0.6522
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6352 - accuracy: 0.7352 - val_loss: 0.6039 - val_accuracy: 0.8546
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5518 - accuracy: 0.8350 - val_loss: 0.5210 - val_accuracy: 0.8546
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4921 - accuracy: 0.8552 - val_loss: 0.4640 - val_accuracy: 0.8546
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4503 - accuracy: 0.8571 - val_loss: 0.4296 - val_accuracy: 0.8546
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4362 - accuracy: 0.8578 - val_loss: 0.4068 - val_accuracy: 0.8546
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8567 - val_loss: 0.3900 - val_accuracy: 0.8546
Epoch 8/200
2

Epoch 59/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3100 - accuracy: 0.8897 - val_loss: 0.2591 - val_accuracy: 0.8996
Epoch 60/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3143 - accuracy: 0.8852 - val_loss: 0.2605 - val_accuracy: 0.9010
Epoch 61/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3040 - accuracy: 0.8871 - val_loss: 0.2606 - val_accuracy: 0.8996
Epoch 62/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2991 - accuracy: 0.8916 - val_loss: 0.2546 - val_accuracy: 0.8996
Epoch 63/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2968 - accuracy: 0.8946 - val_loss: 0.2549 - val_accuracy: 0.8981
Epoch 64/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3012 - accuracy: 0.8942 - val_loss: 0.2564 - val_accuracy: 0.8981
Epoch 65/200
27/27 [==============================] - 0s 1ms/step - loss: 0.2998 - accuracy: 0.8938 - val_loss: 0.2589 - val_accuracy: 0.9010
Epoch 

27/27 [==============================] - 0s 2ms/step - loss: 0.2755 - accuracy: 0.9029 - val_loss: 0.2443 - val_accuracy: 0.9130
Epoch 117/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2765 - accuracy: 0.9062 - val_loss: 0.2440 - val_accuracy: 0.9115
Epoch 118/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2811 - accuracy: 0.9036 - val_loss: 0.2443 - val_accuracy: 0.9115
Epoch 119/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2923 - accuracy: 0.8995 - val_loss: 0.2452 - val_accuracy: 0.9085
Epoch 120/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2764 - accuracy: 0.9051 - val_loss: 0.2451 - val_accuracy: 0.9100
Epoch 121/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2761 - accuracy: 0.9059 - val_loss: 0.2441 - val_accuracy: 0.9115
Epoch 122/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2895 - accuracy: 0.9010 - val_loss: 0.2473 - val_accuracy: 0.9130
Epoch 123/200

27/27 [==============================] - 0s 2ms/step - loss: 0.2535 - accuracy: 0.9145 - val_loss: 0.2426 - val_accuracy: 0.9130
Epoch 174/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2681 - accuracy: 0.9077 - val_loss: 0.2404 - val_accuracy: 0.9115
Epoch 175/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2630 - accuracy: 0.9137 - val_loss: 0.2465 - val_accuracy: 0.9145
Epoch 176/200
27/27 [==============================] - 0s 1ms/step - loss: 0.2646 - accuracy: 0.9096 - val_loss: 0.2426 - val_accuracy: 0.9160
Epoch 177/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2542 - accuracy: 0.9171 - val_loss: 0.2435 - val_accuracy: 0.9160
Epoch 178/200
27/27 [==============================] - 0s 1ms/step - loss: 0.2685 - accuracy: 0.9089 - val_loss: 0.2468 - val_accuracy: 0.9100
Epoch 179/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2595 - accuracy: 0.9149 - val_loss: 0.2458 - val_accuracy: 0.9115
Epoch 180/200

In [20]:
ann_model.evaluate(x_train,y_train)

84/84 [==============================] - 0s 682us/step - loss: 0.2429 - accuracy: 0.8980


[0.2429174929857254, 0.8979744911193848]

In [21]:
ann_model.evaluate(x_test,y_test)

21/21 [==============================] - 0s 679us/step - loss: 0.2541 - accuracy: 0.9040


[0.2540603578090668, 0.904047966003418]

### convertiing probabilities to labels

In [54]:
y_pred_train = ann_model.predict(x_train)
for i in range(0,len(y_pred_train)):
    if y_pred_train[i]>=.5:
        y_pred_train[i] = 1
    else:
        y_pred_train[i] = 0

In [55]:
y_pred_test = ann_model.predict(x_test)
for i in range(0,len(y_pred_test)):
    if y_pred_test[i]>=.5:
        y_pred_test[i] = 1
    else:
        y_pred_test[i] = 0
   

In [56]:
my_dict["model"].append('ANN')
my_dict["train accuracy"].append(round(accuracy_score(y_train,y_pred_train)*100,2))
my_dict["test accuracy"].append(round(accuracy_score(y_test,y_pred_test)*100,2))
my_dict["precision"].append(round(precision_score(y_test,y_pred_test)*100,2))
my_dict["recall"].append(round(recall_score(y_test,y_pred_test)*100,2))

### Naive Bayes

In [57]:
gnb = GaussianNB()
# evaluate model
cross_validate_model(gnb)

Accuracy: 0.875 (0.017)


In [58]:
gnb.fit(x_train,y_train)
populate_matrix(gnb)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()', 'ANN', 'GaussianNB()'], 'train accuracy': [86.01, 100.0, 100.0, 98.42, 95.31, 90.96, 86.98], 'test accuracy': [86.51, 93.25, 97.3, 97.75, 92.8, 90.85, 88.31], 'precision': [56.6, 72.81, 98.77, 98.81, 88.89, 92.86, 57.6], 'recall': [30.93, 85.57, 82.47, 85.57, 57.73, 40.21, 74.23]}


In [59]:
# Adding performance metrics of xgboost manually, i implemented the same in seperate file

my_dict["model"].append('XGBClassifier()')
my_dict["train accuracy"].append(97.78)
my_dict["test accuracy"].append(97.8)
my_dict["precision"].append(98.41)
my_dict["recall"].append(86.11)

### Final performance matrix of all the models

In [60]:
pd.DataFrame(my_dict).sort_values(by='test accuracy',ascending=False)

,model,train accuracy,test accuracy,precision,recall
7,XGBClassifier(),97.78,97.80,98.41,86.11
3,GradientBoostingClassifier(),98.42,97.75,98.81,85.57
2,RandomForestClassifier(),100.00,97.30,98.77,82.47
1,DecisionTreeClassifier(),100.00,93.25,72.81,85.57
4,SVC(),95.31,92.80,88.89,57.73
5,ANN,90.96,90.85,92.86,40.21
6,GaussianNB(),86.98,88.31,57.60,74.23
0,LogisticRegression(),86.01,86.51,56.60,30.93
